In [ ]:
import sys
import os

os.chdir(os.path.abspath(".."))
sys.path.append(os.path.abspath(".."))

# EDINET APIで有価証券報告書を取得 → DB登録フロー

以下のノートブックでは、
1. 書類一覧 APIで提出書類一覧を取得
2. 書類取得 APIでCSV/XBRL等をダウンロード
3. ダウンロードしたファイルを展開し、パスを記録
4. PostgreSQLへレコードを登録

の一連を実装します。

# Cell 1: ライブラリのインポート＆定数定義

In [ ]:
# Cell 1: ライブラリのインポートと設定
import os
import zipfile
import logging
from pathlib import Path

import requests
import pandas as pd
from tqdm import tqdm
from psycopg2.extras import execute_values

from db.connection import get_connection  # あなたの connection.py を指すように

# ロギング設定
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)

# EDINET API のベース URL とキー
API_KEY = os.getenv("EDINET_API_KEY")
BASE_URL = "https://api.edinet-fsa.go.jp/api/v2"
SAVE_ROOT = Path("data/raw/edinet")  # ZIP/CSV を展開するルート

# 取得対象の書類タイプ（2:年次, 3:半期, 4:四半期）
DOC_TYPES = ["120", "130", "140"]

# Cell 2: EDINET API 呼び出し関数定義

In [ ]:
# Cell 2: 日付指定でメタデータを取得する関数
def fetch_document_list_for_date(date: str, doc_type: str) -> pd.DataFrame:
    """
    date (YYYY-MM-DD) + doc_type (文字列) で EDINET の documents.json を呼び、
    'results' 部分を DataFrame で返す。見つからなければ空 DF。
    """
    params = {"date": date, "type": doc_type, "resultType": 1}
    headers = {"X-EDINET-APIKEY": API_KEY}
    r = requests.get(f"{BASE_URL}/documents.json", params=params, headers=headers)
    if r.status_code == 200:
        js = r.json()
        return pd.DataFrame(js.get("results", []))
    elif r.status_code == 204:
        # No Content
        return pd.DataFrame()
    else:
        r.raise_for_status()

# Cell 3: データベース登録用ユーティリティ＆テーブル読み込み

In [ ]:
# Cell 3: 期間（日付）をループしてメタデータを収集
start = "2017-01-01"
end = "2025-12-31"

all_docs = []
dates = pd.date_range(start, end, freq="D").astype(str)
for date in tqdm(dates, desc="Fetching EDINET"):
    for t in DOC_TYPES:
        df = fetch_document_list_for_date(date, t)
        if not df.empty:
            df["doc_type"] = t
            all_docs.append(df)

# マージ前の合計
logging.info(f"▶︎ 取得メタデータ日数: {len(all_docs)} chunks")

# Cell 4: 年次＋四半期レポートを日次ループでまとめて取得→登録

In [ ]:
# Cell 4: 取得結果をまとめて重複除去
if not all_docs:
    raise RuntimeError("EDINET メタデータが一件も取得できませんでした。")
df_all = pd.concat(all_docs, ignore_index=True)
logging.info(f"▶︎ マージ後の総件数: {len(df_all):,}")

# docID で重複を除く
df_all = df_all.drop_duplicates(subset="docID")
logging.info(f"▶︎ docID 一意化後件数: {len(df_all):,}")
df_all.head(3)

In [ ]:
# Cell 5: companies マスタと突合、CSV をダウンロードして edinet_filings に登録
def download_csv_and_unpack(doc_id: str, save_dir: Path) -> Path:
    """ZIP 取得 → 解凍 → 最初の CSV ファイルパスを返却"""
    save_dir.mkdir(parents=True, exist_ok=True)
    zippath = save_dir / f"{doc_id}.zip"
    r = requests.get(
        f"{BASE_URL}/documents/{doc_id}",
        params={"type": 5},
        headers={"X-EDINET-APIKEY": API_KEY},
        stream=True,
    )
    r.raise_for_status()
    with open(zippath, "wb") as f:
        for chunk in r.iter_content(1024):
            f.write(chunk)
    with zipfile.ZipFile(zippath) as z:
        members = [n for n in z.namelist() if n.lower().endswith(".csv")]
        z.extractall(save_dir, members)
        return save_dir / members[0]


# DB 接続して companies を読込
conn = get_connection()
cur = conn.cursor()
df_comp = pd.read_sql("SELECT company_id, edinet_code FROM companies", conn)

records = []
for _, row in tqdm(df_all.iterrows(), total=len(df_all), desc="Download & Prepare"):
    doc_id = row["docID"]
    edc = row["edinetCode"]
    # マスタにある企業のみ処理
    match = df_comp[df_comp["edinet_code"] == edc]
    if match.empty:
        continue
    cid = match["company_id"].iloc[0]
    try:
        csv_path = download_csv_and_unpack(
            doc_id, SAVE_ROOT / row["submitDate"][:4] / edc
        )
    except Exception as e:
        logging.warning(f"ZIP 展開失敗: {doc_id} / {e}")
        continue
    records.append(
        (
            cid,
            edc,
            doc_id,
            row["docTypeCode"],  # メタデータ中の書類コード
            row["submitDate"],
            row["docDescription"],
            str(csv_path),
        )
    )

# INSERT
execute_values(
    cur,
    """
    INSERT INTO edinet_filings
      (company_id, edinet_code, doc_id, doc_type_code, submit_date, description, csv_path)
    VALUES %s
    ON CONFLICT (company_id, doc_id) DO NOTHING;
""",
    records,
)

conn.commit()
cur.close()
conn.close()
logging.info(f"✅ 登録完了: {len(records):,} 件")